In [1]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [1]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [2]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [3]:
##curso...pongo el id del proyecto del gcp de prueba q inicie con este mail
# Define project information
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')
model = GenerativeModel("gemini-1.5-flash-002")
model2= GenerativeModel("gemini-1.5-pro-001")

In [5]:
generation_config = GenerationConfig(
    temperature=0,
    top_p=0.0,
    top_k=1,
    candidate_count=1,
    max_output_tokens=8192,
)

In [6]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [12]:
# Consulta SQL
query2 = """select t.player_id, w.wy_id, t.* from ldp-data-wyscout.transfer_market.players_transfers_curated t
left join ldp-data-wyscout.transfer_market.matching_playerids_v4 w on safe_cast(t.player_id as int)=w.tm_id ;"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df1 = client.query(query2).to_dataframe()
    print(df1.head())
except Exception as e:
    print(f"An error occurred: {e}")


  player_id  wy_id row_ind      player_name season transfer_date  \
0     91280  16081       7     Boné Uaferro  03/04    2003-07-01   
1     41323   <NA>       8    Andreas Luthe  04/05    2004-07-01   
2     26171   <NA>      10  Mike Wunderlich  04/05    2004-07-01   
3     26171   <NA>       9  Mike Wunderlich  05/06    2005-07-01   
4    160968  18544       8    Benno Schmitz  01/02    2001-07-01   

        team_from          team_to market_value transfer_value  \
0  Altglien. Jgd.   U. Berlin Yth.           NA              0   
1  VfL Bochum U17   VfL Bochum U19           NA             NA   
2  Vikt. Köln U19    1.FC Köln U19           NA             NA   
3   1.FC Köln U19     1.FC Köln II           NA             NA   
4  Waldperl. Yth.  B. München Yth.           NA              0   

  contract_expiry                                         player_url  \
0              NA  https://www.transfermarkt.com/bone-uaferro/pro...   
1              NA  https://www.transfermarkt.com/a

In [13]:
df1.head()

,player_id,wy_id,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id_1,teams_url,team_id,player_transfer_id
0,91280,16081,7,Boné Uaferro,03/04,2003-07-01,Altglien. Jgd.,U. Berlin Yth.,NA,0,NA,https://www.transfermarkt.com/bone-uaferro/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,91280,https://www.transfermarkt.com/1-fc-saarbrucken...,1,20030701191280
1,41323,<NA>,8,Andreas Luthe,04/05,2004-07-01,VfL Bochum U17,VfL Bochum U19,NA,NA,NA,https://www.transfermarkt.com/andreas-luthe/pr...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,41323,https://www.transfermarkt.com/1-fc-kaiserslaut...,2,20040701241323
2,26171,<NA>,10,Mike Wunderlich,04/05,2004-07-01,Vikt. Köln U19,1.FC Köln U19,NA,NA,NA,https://www.transfermarkt.com/mike-wunderlich/...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,26171,https://www.transfermarkt.com/1-fc-kaiserslaut...,2,20040701226171
3,26171,<NA>,9,Mike Wunderlich,05/06,2005-07-01,1.FC Köln U19,1.FC Köln II,NA,NA,NA,https://www.transfermarkt.com/mike-wunderlich/...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,26171,https://www.transfermarkt.com/1-fc-kaiserslaut...,2,20050701226171
4,160968,18544,8,Benno Schmitz,01/02,2001-07-01,Waldperl. Yth.,B. München Yth.,NA,0,NA,https://www.transfermarkt.com/benno-schmitz/pr...,2024-09-07,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,160968,https://www.transfermarkt.com/1-fc-koln/starts...,3,200107013160968


In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104606 entries, 0 to 1104605
Data columns (total 19 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   player_id           1104605 non-null  object             
 1   wy_id               579565 non-null   Int64              
 2   row_ind             1104606 non-null  object             
 3   player_name         1104606 non-null  object             
 4   season              1104606 non-null  object             
 5   transfer_date       1104606 non-null  object             
 6   team_from           1104606 non-null  object             
 7   team_to             1104606 non-null  object             
 8   market_value        1104606 non-null  object             
 9   transfer_value      1104606 non-null  object             
 10  contract_expiry     1104606 non-null  object             
 11  player_url          1104606 non-null  object             
 12  

In [ ]:
dftext = df1.to_csv(index=False)

In [14]:
###ME TRAIGO LOS DATOS DE LA TABLA DE CAPOLOGY
query = """select w.playerId, t.tm_id, w.* from ldp-data-wyscout.wyscout_curated.transfer_con_nombre w
left join ldp-data-wyscout.transfer_market.matching_playerids_v4 t on w.playerId=t.wy_id;
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df2 = client.query(query).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")

   playerId  tm_id                    nombre  transferId  fromTeamId  \
0   1061989   <NA>  Hákon Atli Aðalsteinsson     2252367        7724   
1   1061951   <NA>                 Zen Galea     2252155        <NA>   
2   1061960   <NA>            Daniel Radeski     2252179        <NA>   
3   1061902   <NA>            Sanel Ibisevic     2251976       34765   
4   1061905   <NA>     Hassan Mushtaq Taheri     2252004        7060   

           fromTeamName  toTeamId             toTeamName active   startDate  \
0           KA Akureyri      7749              Völsungur      0  2023-04-06   
1                            <NA>        Mgarr United FC      0  2024-03-01   
2                            8380  Stirling Macedonia SC      0  2024-03-01   
3  TSV Havelse Under 19      7143               Follo FK      0  2018-09-04   
4          Boo FF Nacka      7143               Follo FK      0  2022-09-08   

      endDate      type value currency announceDate  playerId_1  
0  2024-02-02      Loan   

In [17]:
df2.head(20)

,playerId,tm_id,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId_1
0,1061989,<NA>,Hákon Atli Aðalsteinsson,2252367,7724,KA Akureyri,7749,Völsungur,0,2023-04-06,2024-02-02,Loan,0,EUR,,1061989
1,1061951,<NA>,Zen Galea,2252155,<NA>,,<NA>,Mgarr United FC,0,2024-03-01,None,Unknown,0,EUR,,1061951
2,1061960,<NA>,Daniel Radeski,2252179,<NA>,,8380,Stirling Macedonia SC,0,2024-03-01,None,Unknown,0,EUR,,1061960
3,1061902,<NA>,Sanel Ibisevic,2251976,34765,TSV Havelse Under 19,7143,Follo FK,0,2018-09-04,None,Transfer,0,EUR,,1061902
4,1061905,<NA>,Hassan Mushtaq Taheri,2252004,7060,Boo FF Nacka,7143,Follo FK,0,2022-09-08,None,Transfer,0,EUR,,1061905
5,1061911,<NA>,Hans Kristian Heier,2252032,7191,Lillestrøm SK II,7143,Follo FK,0,2023-08-21,None,Transfer,0,EUR,,1061911
6,1061921,<NA>,Herman Oen Paulsrud,2252070,24654,Grorud Idrettslag II,7345,Grorud IL,0,2024-03-22,None,Transfer,0,EUR,,1061921
7,1061847,<NA>,Lucas Poulsen,2352906,7463,BK Frem 1886,34555,Sundby BK,0,2024-08-01,None,Transfer,0,EUR,,1061847
8,1061847,<NA>,Lucas Poulsen,2251604,34555,Sundby BK,7463,BK Frem 1886,0,2024-02-01,2024-06-30,Transfer,0,EUR,,1061847
9,1061985,<NA>,Kasper Sommerstedt,2408653,11173,ÍF Fuglafjørdur,22910,Hedensted BK,0,2024-08-27,None,Transfer,0,EUR,,1061985


In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 16 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   playerId      Int64 
 1   tm_id         Int64 
 2   nombre        object
 3   transferId    Int64 
 4   fromTeamId    Int64 
 5   fromTeamName  object
 6   toTeamId      Int64 
 7   toTeamName    object
 8   active        object
 9   startDate     object
 10  endDate       object
 11  type          object
 12  value         object
 13  currency      object
 14  announceDate  object
 15  playerId_1    Int64 
dtypes: Int64(6), object(10)
memory usage: 247.1+ MB


In [25]:
#df1.dropna(inplace=True)

In [13]:
dftext2 = df2.to_csv(index=False)

In [30]:
query = """WITH tabla1 AS (
    -- Primera consulta: renombramos los playerId y tm_id para evitar ambigüedad
    SELECT w.playerId AS wy_playerId, t.tm_id AS tm_id, w.*
    FROM ldp-data-wyscout.wyscout_curated.transfer_con_nombre w
    LEFT JOIN ldp-data-wyscout.transfer_market.matching_playerids_v4 t 
    ON w.playerId = t.wy_id
),
tabla2 AS (
    -- Segunda consulta: renombramos los player_id y wy_id para evitar ambigüedad
    SELECT t.player_id AS tm_playerId, w.wy_id AS wy_id, t.*
    FROM ldp-data-wyscout.transfer_market.players_transfers_curated t
    LEFT JOIN ldp-data-wyscout.transfer_market.matching_playerids_v4 w 
    ON SAFE_CAST(t.player_id AS INT) = w.tm_id
)

-- Unir ambas tablas por los IDs correspondientes
SELECT  *
FROM tabla1
JOIN tabla2 
ON tabla1.wy_playerId = tabla2.wy_id 
  AND tabla1.tm_id =  SAFE_CAST(tabla2.tm_playerId AS INT);
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df3 = client.query(query).to_dataframe()
    print(df3.head())
except Exception as e:
    print(f"An error occurred: {e}")

   wy_playerId   tm_id                  nombre  transferId  fromTeamId  \
0       240902  104871    Martín Ariel Pucheta     1895516       12264   
1       240902  104871    Martín Ariel Pucheta        6559       23216   
2       240902  104871    Martín Ariel Pucheta      514314       17383   
3       805662  976205  Daniel Atanes Rekelhof     2199128       39691   
4       851642  701527     Jose  Barbosa Nhoca     1731386        <NA>   

                     fromTeamName  toTeamId                      toTeamName  \
0  Club Atlético Sarmiento de Res     12066     Club Olimpo de Bahía Blanca   
1                  Andes Talleres     12132         Independiente Rivadavia   
2                      CD Motagua     23214  Club Atlético San Lorenzo de A   
3                      UD Montijo       731                  Rayo Cantabria   
4                                        53                       Noordwijk   

  active   startDate  ... transfer_value contract_expiry  \
0      0  2023-01-05

In [31]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560963 entries, 0 to 4560962
Data columns (total 35 columns):
 #   Column              Dtype              
---  ------              -----              
 0   wy_playerId         Int64              
 1   tm_id               Int64              
 2   nombre              object             
 3   transferId          Int64              
 4   fromTeamId          Int64              
 5   fromTeamName        object             
 6   toTeamId            Int64              
 7   toTeamName          object             
 8   active              object             
 9   startDate           dbdate             
 10  endDate             dbdate             
 11  type                object             
 12  value               object             
 13  currency            object             
 14  announceDate        object             
 15  playerId            Int64              
 16  tm_playerId         object             
 17  wy_id               Int64  

In [32]:
# Contar los valores nulos en cada columna
null_counts = df3.isnull().sum()

# Filtrar las columnas que tienen valores nulos
columns_with_nulls = null_counts[null_counts > 0]

print(columns_with_nulls)

fromTeamId      528665
fromTeamName    329914
toTeamId        213755
toTeamName      163435
startDate           97
endDate         787223
type                41
currency           449
blob            378597
dtype: int64


In [23]:
# Obtener el rango de fechas en df3
fecha_inicio = df3['startDate'].min()
fecha_fin = df3['endDate'].max()

print(f"El rango de fechas en df3 es de {fecha_inicio} a {fecha_fin}")

# Verificar si hay valores repetidos en df3
valores_repetidos = df3.duplicated().sum()

print(f"El número de valores repetidos en df3 es: {valores_repetidos}")

El rango de fechas en df3 es de 1984-02-14 a 2030-06-30
El número de valores repetidos en df3 es: 0


In [35]:
df4=df3.dropna()

In [34]:
df3.head(50)

,wy_playerId,tm_id,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,...,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,240902,104871,Martín Ariel Pucheta,1895516,12264,Club Atlético Sarmiento de Res,12066,Club Olimpo de Bahía Blanca,0,2023-01-05,...,0,NA,https://www.transfermarkt.com/martin-pucheta/p...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,104871,https://www.transfermarkt.com/club-olimpo/star...,7468,202107017468104871
1,240902,104871,Martín Ariel Pucheta,6559,23216,Andes Talleres,12132,Independiente Rivadavia,0,2013-07-05,...,0,NA,https://www.transfermarkt.com/martin-pucheta/p...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,104871,https://www.transfermarkt.com/club-olimpo/star...,7468,202107017468104871
2,240902,104871,Martín Ariel Pucheta,514314,17383,CD Motagua,23214,Club Atlético San Lorenzo de A,1,2017-07-01,...,0,NA,https://www.transfermarkt.com/martin-pucheta/p...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,104871,https://www.transfermarkt.com/club-olimpo/star...,7468,202107017468104871
3,805662,976205,Daniel Atanes Rekelhof,2199128,39691,UD Montijo,731,Rayo Cantabria,0,2024-01-24,...,0,2024-06-30,https://www.transfermarkt.com/dani-atanes/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,976205,https://www.transfermarkt.com/vv-noordwijk/sta...,7474,202401247474976205
4,851642,701527,Jose Barbosa Nhoca,1731386,<NA>,,53,Noordwijk,0,2022-07-01,...,0,NA,https://www.transfermarkt.com/jose-barbosa-nho...,2024-09-11,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,701527,https://www.transfermarkt.com/vv-noordwijk/sta...,7474,201807017474701527
5,876913,1040582,Georgios Lesaj,2056948,71504,AE Iraklis Larissa,12604,Diagoras Rodou FC,0,2023-08-05,...,0,NA,https://www.transfermarkt.com/georgios-lleshaj...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1040582,https://www.transfermarkt.com/iraklis-larisas/...,7476,2021070174761040582
6,267068,269779,Mauricio Sebastián Gómez Castro,1396270,<NA>,None,15637,Montevideo Wanderers Fútbol Cl,0,2013-05-01,...,0,NA,https://www.transfermarkt.com/mauricio-gomez/p...,2023-10-03,2023-09-30 17:49:27+00:00,None,269779,https://www.transfermarkt.com/miramar-misiones...,7477,201507017477269779
7,388825,399796,Mathías Fernando Rodríguez Laites,1608868,<NA>,None,15638,Club Sportivo Miramar Misiones,0,2022-02-07,...,NA,2016-06-30,https://www.transfermarkt.com/mathias-rodrigue...,2024-09-07,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,399796,https://www.transfermarkt.com/miramar-misiones...,7477,201604017477399796
8,748875,886885,Brian Cordara Damasco,2202173,15638,Club Sportivo Miramar Misiones,15662,La Luz FC,0,2024-01-26,...,0,2022-12-31,https://www.transfermarkt.com/brian-cordara/pr...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,886885,https://www.transfermarkt.com/miramar-misiones...,7477,202206237477886885
9,748875,886885,Brian Cordara Damasco,1545565,937,Zamudio SD,15639,Club Atlético Rentistas,0,2020-01-01,...,0,2022-12-31,https://www.transfermarkt.com/brian-cordara/pr...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,886885,https://www.transfermarkt.com/miramar-misiones...,7477,202206237477886885


In [36]:
df4.head()

,wy_playerId,tm_id,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,...,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,240902,104871,Martín Ariel Pucheta,1895516,12264,Club Atlético Sarmiento de Res,12066,Club Olimpo de Bahía Blanca,0,2023-01-05,...,0,NA,https://www.transfermarkt.com/martin-pucheta/p...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,104871,https://www.transfermarkt.com/club-olimpo/star...,7468,202107017468104871
1,240902,104871,Martín Ariel Pucheta,6559,23216,Andes Talleres,12132,Independiente Rivadavia,0,2013-07-05,...,0,NA,https://www.transfermarkt.com/martin-pucheta/p...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,104871,https://www.transfermarkt.com/club-olimpo/star...,7468,202107017468104871
2,240902,104871,Martín Ariel Pucheta,514314,17383,CD Motagua,23214,Club Atlético San Lorenzo de A,1,2017-07-01,...,0,NA,https://www.transfermarkt.com/martin-pucheta/p...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,104871,https://www.transfermarkt.com/club-olimpo/star...,7468,202107017468104871
5,876913,1040582,Georgios Lesaj,2056948,71504,AE Iraklis Larissa,12604,Diagoras Rodou FC,0,2023-08-05,...,0,NA,https://www.transfermarkt.com/georgios-lleshaj...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1040582,https://www.transfermarkt.com/iraklis-larisas/...,7476,2021070174761040582
9,748875,886885,Brian Cordara Damasco,1545565,937,Zamudio SD,15639,Club Atlético Rentistas,0,2020-01-01,...,0,2022-12-31,https://www.transfermarkt.com/brian-cordara/pr...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,886885,https://www.transfermarkt.com/miramar-misiones...,7477,202206237477886885


In [37]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2940696 entries, 0 to 4560962
Data columns (total 35 columns):
 #   Column              Dtype              
---  ------              -----              
 0   wy_playerId         Int64              
 1   tm_id               Int64              
 2   nombre              object             
 3   transferId          Int64              
 4   fromTeamId          Int64              
 5   fromTeamName        object             
 6   toTeamId            Int64              
 7   toTeamName          object             
 8   active              object             
 9   startDate           dbdate             
 10  endDate             dbdate             
 11  type                object             
 12  value               object             
 13  currency            object             
 14  announceDate        object             
 15  playerId            Int64              
 16  tm_playerId         object             
 17  wy_id               Int64       

In [38]:
df4.tail()

,wy_playerId,tm_id,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,...,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
4560958,215333,245315,Eric Miller,59943,8142,Montreal Impact,7853,Colorado Rapids,1,2016-02-14,...,NA,NA,https://www.transfermarkt.com/eric-miller/prof...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,245315,https://www.transfermarkt.com/portland-timbers...,4291,201911264291245315
4560959,216964,231923,Aljaž Ivačič,502103,12004,NK Radomlje,11953,Olimpija,1,2017-08-31,...,NA,2025-12-31,https://www.transfermarkt.com/aljaz-ivacic/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,231923,https://www.transfermarkt.com/portland-timbers...,4291,202403194291231923
4560960,90838,165361,Yimmi Javier Chará Zamora,296470,15465,CSyD Dorados de Sinaloa,15444,CF Monterrey,0,2016-06-30,...,NA,2015-12-31,https://www.transfermarkt.com/yimmi-chara/prof...,2024-09-11,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,165361,https://www.transfermarkt.com/portland-timbers...,4291,201512314291165361
4560961,90838,165361,Yimmi Javier Chará Zamora,489236,15444,CF Monterrey,12391,Club Atlético Nacional SA,0,2015-07-01,...,NA,2015-12-31,https://www.transfermarkt.com/yimmi-chara/prof...,2024-09-11,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,165361,https://www.transfermarkt.com/portland-timbers...,4291,201512314291165361
4560962,351808,344015,Ben Woodburn,1422599,1612,Liverpool FC,8510,Heart of Midlothian FC,0,2021-08-23,...,NA,2022-06-30,https://www.transfermarkt.com/ben-woodburn/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,344015,https://www.transfermarkt.com/preston-north-en...,466,20180801466344015


In [39]:
# Obtener el rango de fechas en df4
fecha_inicio_df4 = df4['startDate'].min()
fecha_fin_df4 = df4['endDate'].max()

print(f"El rango de fechas en df4 es de {fecha_inicio_df4} a {fecha_fin_df4}")

El rango de fechas en df4 es de 1996-07-01 a 2030-06-30


In [40]:
# Realizar un merge de df3 y df4 con indicador para identificar las filas que no coinciden
merged_df = df3.merge(df4, on=['wy_ adplayerId', 'tm_id'], how='left', indicator=True)

# Filtrar las filas que están solo en df3
not_in_df4 = merged_df[merged_df['_merge'] == 'left_only']

# Eliminar la columna '_merge' ya que no es necesaria
not_in_df4 = not_in_df4.drop(columns=['_merge'])

# Mostrar los resultados
print(not_in_df4)

MemoryError: Unable to allocate 107. GiB for an array with shape (24, 596066062) and data type object

In [16]:
dftext2 = df2.to_csv(index=False)

In [39]:
prompt1= """Que informacion vez en estos textos?"""

In [40]:
##salida armoa
def generate():
  vertexai.init(project=PROJECT_ID, location=LOCATION)
  responses = model.generate_content(
      [dftext, dftext2, prompt1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")


generate()

These texts contain data about football (soccer) clubs across various leagues and countries.  The information includes:

* **Geographic Location:** Region (e.g., Europe, South America), Country (e.g., Spain, Brazil), and sometimes more specific area names.

* **League and Club Information:** The name of the league (e.g., La Liga, MLS) and the specific football club (e.g., Real Madrid, Atlanta United).  There's also a division level specified for each domestic competition.

* **Attendance Data:** Weekly average attendance and annual average attendance figures.

* **Currency:** The currency in which the attendance figures are reported (e.g., euros, dollars).

* **Unique Identifiers:**  `wyId` for some entries, providing a unique identifier for each club.  There's also a `name` and `officialName` for each club.


In short, the data represents a dataset suitable for analysis of football club attendance across different leagues and regions.  The inclusion of `wyId` suggests the data might b

In [18]:
prompt2= """Sos un agente que analiza datos tu funcion es generar una unica tabla a partir de la informacion que te doy en 2 csv, que corresponden a 2 tablas.
Estos textos contienen informacion de dos tablas:
El primer texto contiene la informacion de la primera tabla que contiene las siguientes columnas: 	
-region: corresponde a la region de donde es el club
-pais: corresponde al pais de donde es el club	
-liga: corresponde a la liga del club	
-club: corresponde al nombre del club	
-weeklyavg:	corresponde al salario semanal promedio por jugador del club
-annualavg: corresponde al salario anual promedio por jugador del club	
-moneda: corresponde a la moneda en la que se mide el salario
Y tiene 570 filas
El segundo texto contiene la informacion de la segunda tabla que contiene las siguientes columnas:
-wyId: corresponde al id del club	
-name: corresponde al nombre del club	
-officialName: corresponde al nombre oficial del club, puede ser levemente diferente al nombre o contenter alguna palabra mas.	
-area_name: corresponde al pais de donde es el club
-domestic_competition_name: corresponde a la liga del club		
-region: corresponde a la region de donde es el club
y contiene 2000 filas
La infomacion en ambas tablas es muy similar. La tabla 2 deberia tener todos los clubes de la tabla 1, no asi al revez. La tablta 1 tiene muchas menos filas que la tabla dos. 
El objetivo es encontrar el id de los clubes de la tabla 1, en la tabla 2 y crear una tabla final que contenga la informacion de la tabla 1 pero con wl wyId de la tabla dos.
La tabla final tiene que contenter la union de estas dos tablas con la informacion de ambas tablas, pero solo con los clubes que estan en ambas tablas.
Es decir solamente tienen que quedar en el resultado, los valores que coinciden dentro de las dos tablas con su ID, deberian ser la misma cantidad de valores que la primera tabla
Quiero que vos hagas la tabla. No quiero que me des el codigo de SQL para hacerlo yo.
Quiero que lo hagas con todos los valores de la tabla uno hasta terminar de leer ambos textos.
Es decir necesito que me devuelvas los 570 clubes del dftext con su wyid en una tabla.
La respuesta tiene que ser unicamente esta tabla de 570 filas con las columnas  region | pais | liga | club | weeklyavg | annualavg | moneda | domestic_competition_divisionLevel | wyId | name | officialName | area_name | domestic_competition_name 
"""

In [66]:

def generate():
  vertexai.init(project=PROJECT_ID, location=LOCATION)
  model3 = GenerativeModel("gemini-1.5-pro-preview-0409")
  responses = model3.generate_content(
      [dftext, dftext2, prompt2],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )
    # Captura el texto completo en una variable
  full_text = ""
  for response in responses:
      full_text += response.text

  return full_text

# Ejecuta la función y captura el texto generado
result_text = generate()

print(result_text)

```
region | pais | liga | club | weeklyavg | annualavg | moneda | domestic_competition_divisionLevel | wyId | name | officialName | area_name | domestic_competition_name
------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | --------
Central/North America | United States | MLS | Atlanta United | 6451 | 335452 | dolares | 1 | 58165 | Atlanta United | Atlanta United FC | United States | MLS
Central/North America | United States | MLS | New England Revolution | 9209 | 478875 | dolares | 1 | 7854 | New England | New England Revolution | United States | MLS
Central/North America | United States | MLS | Philadelphia Union | 6990 | 363486 | dolares | 1 | 8061 | Philadelphia Union | Philadelphia Union | United States | MLS
Central/North America | United States | MLS | Seattle Sounders | 10668 | 554728 | dolares | 1 | 8067 | Seattle Sounders | Seattle Sounders FC | United States | MLS
Central/North America | United Sta

bucle pero da medio desordenado

In [ ]:
def generate():
    vertexai.init(project=PROJECT_ID, location=LOCATION)
    model3 = GenerativeModel("gemini-1.5-pro-preview-0409")
    
    # Inicializamos un conjunto para almacenar filas únicas
    unique_rows = set()
    
    while len(unique_rows) < 570:
        # Realizamos la llamada al modelo
        responses = model3.generate_content(
            [dftext, dftext2, prompt2],
            generation_config=generation_config,
            safety_settings=safety_settings,
            stream=True,
        )
        
        for response in responses:
            # Dividimos la respuesta generada en filas
            rows = response.text.strip().split('\n')
            
            # Añadimos las filas al conjunto para eliminar duplicados
            unique_rows.update(rows)

        # Verificamos si ya tenemos 570 filas únicas
        if len(unique_rows) >= 570:
            break
    
    # Unimos las filas únicas en un solo texto
    full_text = "\n".join(unique_rows)

    return full_text


pruebo con el mismo prompt pero con un bucle
ES EL QUE FUNCIONA!!!! VA ITERANDO DE A 100 FILAS, LA VARIABLE QUE ITERO ESTA EN DATAFRAME, LA QUE LE PASO PARA COMPARAR ESTA EN CSV

In [19]:
import pandas as pd

def generate():
    vertexai.init(project=PROJECT_ID, location=LOCATION)
    model3 = GenerativeModel("gemini-1.5-pro-preview-0409")
    
    # Lee el archivo CSV como un DataFrame
    df1 #df1 = pd.read_csv('ruta/al/archivo/df1.csv')  # Reemplaza con la ruta correcta
    df2 #dftext2 = pd.read_csv('ruta/al/archivo/dftext2.csv')  # Reemplaza con la ruta correcta

    # Inicializamos variables para acumular el texto
    full_text = ""
    total_rows = df1.shape[0]
    batch_size = 100  # Tamaño del lote

    # Itera sobre el DataFrame en lotes
    for i in range((total_rows + batch_size - 1) // batch_size):  # Asegura que se cubran todos los lotes
        batch = df1[i * batch_size:(i + 1) * batch_size]

        # Convierte el lote a un formato de texto (CSV sin encabezados)
        batch_text = batch.to_csv(index=False, header=False)

        # Realiza la llamada al modelo con el lote actual
        responses = model3.generate_content(
            [batch_text, dftext2, prompt2],  # Asegúrate de que df12 esté en el formato correcto
            generation_config=generation_config,
            safety_settings=safety_settings,
            stream=True,
        )
        
        # Captura las respuestas generadas
        for response in responses:
            full_text += response.text

    return full_text

# Llama a la función para generar el texto
result_text = generate()



In [20]:
print(result_text)

```
| region | pais | liga | club | weeklyavg | annualavg | moneda | domestic_competition_divisionLevel | wyId | name | officialName | area_name | domestic_competition_name |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Central/North America | United States | MLS | Atlanta United | 6451 | 335452 | dolares | 1 | 58165 | Atlanta United | Atlanta United FC | United States | MLS |
| Central/North America | United States | MLS | New England Revolution | 9209 | 478875 | dolares | 1 | 7854 | New England | New England Revolution | United States | MLS |
| Central/North America | United States | MLS | Philadelphia Union | 6990 | 363486 | dolares | 1 | 8061 | Philadelphia Union | Philadelphia Union | United States | MLS |
| Central/North America | United States | MLS | Seattle Sounders | 10668 | 554728 | dolares | 1 | 8067 | Seattle Sounders | Seattle Sounders FC | United States | MLS |
| Central/North America | United States | MLS | Nashville SC | 11651 | 605862 | dolares | 1 | 30149 

In [21]:
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', result_text)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
dfunion = pd.DataFrame(matches)#, columns=[club | liga | pais | wyId | officialName | name | area_name | domestic_competition_name ])

dfunion = dfunion[~dfunion.eq('---').all(axis=1)]
dfunion.columns = dfunion.iloc[0]  # La primera fila se convierte en nombres de columna
dfunion = dfunion[1:]
# Resetea el índice
dfunion.reset_index(drop=True, inplace=True)

print(dfunion)

In [2]:
dfunion.head()

NameError: name 'dfunion' is not defined

In [1]:
dfunion.info()

NameError: name 'dfunion' is not defined

In [34]:
# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.otras_fuentes.capology_gemini'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(dfunion, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.otras_fuentes.capology_gemini.


In [ ]:
prompt=f"""Sos un agente que analiza datos tu funcion es generar una unica tabla a partir de la informacion que te doy en 2 csv, que corresponden a 2 tablas.
Estos textos contienen informacion de dos tablas:
El primer texto:{dftext} contiene la informacion de la primera tabla que contiene las siguientes columnas: 	
-region: corresponde a la region de donde es el club
-pais: corresponde al pais de donde es el club	
-liga: corresponde a la liga del club	
-club: corresponde al nombre del club	
-weeklyavg:	corresponde al salario semanal promedio por jugador del club
-annualavg: corresponde al salario anual promedio por jugador del club	
-moneda: corresponde a la moneda en la que se mide el salario
El segundo texto: {dftext2} contiene la informacion de la segunda tabla que contiene las siguientes columnas:
-wyId: corresponde al id del club	
-name: corresponde al nombre del club	
-officialName: corresponde al nombre oficial del club, puede ser levemente diferente al nombre o contenter alguna palabra mas.	
-area_name: corresponde al pais de donde es el club
-domestic_competition_name: corresponde a la liga del club		
-region: corresponde a la region de donde es el club
La infomacion en ambas tablas es muy similar. La tabla 2 deberia tener todos los clubes de la tabla 1, no asi al revez. La tablta 1 tiene muchas menos filas que la tabla dos. 
El objetivo es encontrar el id de los clubes de la tabla 1, en la tabla 2. 
La tabla final tiene que contenter la union de estas dos tablas con la informacion de ambas tablas, pero solo con los clubes que estan en ambas tablas.
Es decir solamente tienen que quedar en el resultado, los valores que coinciden dentro de las dos tablas con su ID, deberian ser la misma cantidad de valores que la primera tabla
Quiero que vos hagas la tabla. No quiero que me des el codigo de SQL para hacerlo yo."""

responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Inicializar una variable para almacenar el resultado completo
full_response = ""

# Recopilar y unir todas las partes de la respuesta
for response in responses:
    full_response += response.text

# Imprimir o usar la respuesta completa almacenada en la variable
print(full_response)

In [ ]:
pattern = r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|'
matches = re.findall(pattern, full_response)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
dfunion = pd.DataFrame(matches)#, columns=[club | liga | pais | wyId | officialName | name | area_name | domestic_competition_name ])

dfunion = dfunion[~dfunion.eq('---').all(axis=1)]
dfunion.columns = dfunion.iloc[0]  # La primera fila se convierte en nombres de columna
dfunion = dfunion[1:]
# Resetea el índice
dfunion.reset_index(drop=True, inplace=True)

print(dfunion)

In [65]:
prompt22=f"""Sos un agente que analiza datos tu funcion es generar una unica tabla a partir de la informacion que te doy en 2 csv, que corresponden a 2 tablas.
Estos textos contienen informacion de dos tablas:
El primer texto:{dftext} contiene la informacion de la primera tabla que contiene las siguientes columnas: 	
-region: corresponde a la region de donde es el club
-pais: corresponde al pais de donde es el club	
-liga: corresponde a la liga del club	
-club: corresponde al nombre del club	
-weeklyavg:	corresponde al salario semanal promedio por jugador del club
-annualavg: corresponde al salario anual promedio por jugador del club	
-moneda: corresponde a la moneda en la que se mide el salario
El segundo texto: {dftext2} contiene la informacion de la segunda tabla que contiene las siguientes columnas:
-wyId: corresponde al id del club	
-name: corresponde al nombre del club	
-officialName: corresponde al nombre oficial del club, puede ser levemente diferente al nombre o contenter alguna palabra mas.	
-area_name: corresponde al pais de donde es el club
-domestic_competition_name: corresponde a la liga del club		
-region: corresponde a la region de donde es el club
La infomacion en ambas tablas es muy similar. La tabla 2 deberia tener todos los clubes de la tabla 1, no asi al revez. La tablta 1 tiene muchas menos filas que la tabla dos. 
El objetivo es encontrar el id de los clubes de la tabla 1, en la tabla 2. 
La tabla final tiene que contenter la union de estas dos tablas con la informacion de ambas tablas, pero solo con los clubes que estan en la primera tabla. Tiene que haber aproximadamente 570 clubes.
Es decir solamente tienen que quedar en el resultado, los valores que coinciden dentro de las dos tablas con su ID, deberian ser la misma cantidad de valores que la primera tabla
Quiero que vos hagas la tabla. No quiero que me des el codigo de SQL para hacerlo yo.
Si hay valores similares unificalos, por ejemplo region de ambas tablas, pais de una con area_name de otra, club con name y officialName, liga con domestic_competition_name"""

responses2 = model.generate_content(prompt22, generation_config=generation_config, stream=True)

# Inicializar una variable para almacenar el resultado completo
full_response2 = ""

# Recopilar y unir todas las partes de la respuesta
for response in responses2:
    full_response2 += response.text

# Imprimir o usar la respuesta completa almacenada en la variable
print(full_response2)

## Tabla Combinada de Clubes de Fútbol

| wyId | region | pais | liga | club | weeklyavg | annualavg | moneda | officialName | area_name | domestic_competition_name |
|---|---|---|---|---|---|---|---|---|---|---|
| 12397 | South America | Colombia | Liga BetPlay | Pasto | 3314 | 172308 | dolares | Asociación Deportivo Pasto | Colombia | Liga BetPlay |
| 12405 | South America | Colombia | Liga BetPlay | Envigado | 3183 | 165517 | dolares | Envigado FC | Colombia | Liga BetPlay |
| 12392 | South America | Colombia | Liga BetPlay | Once Caldas | 3762 | 195625 | dolares | Once Caldas SA | Colombia | Liga BetPlay |
| 12425 | South America | Colombia | Liga BetPlay | Alianza | 3293 | 171250 | dolares | Alianza FC | Colombia | Liga BetPlay |
| 12421 | South America | Colombia | Liga BetPlay | Patriotas Boyacá | 2543 | 132258 | dolares | Corporación Deportiva Patriotas FC | Colombia | Liga BetPlay |
| 12413 | South America | Colombia | Liga BetPlay | Águilas Doradas | 3036 | 157857 | dolares |

In [66]:
pattern2 = r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|'
matches2 = re.findall(pattern2, full_response2)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
dfunion2 = pd.DataFrame(matches2)#, columns=[club | liga | pais | wyId | officialName | name | area_name | domestic_competition_name ])

dfunion2 = dfunion2[~dfunion2.eq('---').all(axis=1)]
dfunion2.columns = dfunion2.iloc[0]  # La primera fila se convierte en nombres de columna
dfunion2 = dfunion2[1:]
# Resetea el índice
dfunion2.reset_index(drop=True, inplace=True)

dfunion2.head()

,wyId,region,pais,liga,club,weeklyavg,annualavg,moneda,officialName,area_name,domestic_competition_name
0,12397,South America,Colombia,Liga BetPlay,Pasto,3314,172308,dolares,Asociación Deportivo Pasto,Colombia,Liga BetPlay
1,12405,South America,Colombia,Liga BetPlay,Envigado,3183,165517,dolares,Envigado FC,Colombia,Liga BetPlay
2,12392,South America,Colombia,Liga BetPlay,Once Caldas,3762,195625,dolares,Once Caldas SA,Colombia,Liga BetPlay
3,12425,South America,Colombia,Liga BetPlay,Alianza,3293,171250,dolares,Alianza FC,Colombia,Liga BetPlay
4,12421,South America,Colombia,Liga BetPlay,Patriotas Boyacá,2543,132258,dolares,Corporación Deportiva Patriotas FC,Colombia,Liga BetPlay


In [67]:
dfunion2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   wyId                       185 non-null    object
 1   region                     185 non-null    object
 2   pais                       185 non-null    object
 3   liga                       185 non-null    object
 4   club                       185 non-null    object
 5   weeklyavg                  185 non-null    object
 6   annualavg                  185 non-null    object
 7   moneda                     185 non-null    object
 8   officialName               185 non-null    object
 9   area_name                  185 non-null    object
 10  domestic_competition_name  185 non-null    object
dtypes: object(11)
memory usage: 16.0+ KB


ya no me da mas bien lo q me daba bien antes

In [69]:
prompt5 = f"""Sos un agente que analiza datos necesito que generes una tabla con la union de la informacion de dos tablas, a partir del analisis de 2 tablas.
La primera tabla contiene las columnas region, club, liga, pais de 570 clubes y la segunda tabla contiene las columnas  wyId, officialName,name, area_name y comestic_competition_name con 28425 valores. Lo que yo necesito que hagas es, encontrar las coincidencias
entre todos los valores de las columnas y a partir de esas coincidencias obtener el valor del id en el campo "wyId" de la segunda tabla
estas son las tablas: {dftext} y {dftext2}
Es decir solamente tienen que quedar en el resultado, los valores que coinciden dentro de las dos tablas con su ID, deberian ser la misma cantidad de valores que la tabla inicial
   """
# Generar la respuesta utilizando el prompt actualizado
responses5 = model.generate_content(prompt5, generation_config=generation_config, stream=True)

# Inicializar una variable para almacenar el resultado completo
full_response5 = ""

# Recopilar y unir todas las partes de la respuesta
for response5 in responses5:
    full_response5 += response5.text

# Imprimir o usar la respuesta completa almacenada en la variable
print(full_response5)

Entiendo perfectamente lo que necesitas. Para unir la información de ambas tablas y obtener el "wyId" correspondiente a cada club, necesitas realizar una unión basada en las coincidencias entre los valores de las columnas. 

Sin embargo, para poder generar la tabla de unión, necesito que me proporciones la segunda tabla con los datos de "wyId, officialName, name, area_name y comestic_competition_name". 

Una vez que me proporciones la segunda tabla, puedo realizar la unión y generar la tabla resultante con las columnas:

* **region**
* **pais**
* **liga**
* **club**
* **weeklyavg**
* **annualavg**
* **moneda**
* **wyId**

**Ejemplo de cómo se vería la tabla resultante:**

| region | pais | liga | club | weeklyavg | annualavg | moneda | wyId |
|---|---|---|---|---|---|---|---|
| Asia | Arabia Saudita | Saudi Pro League | Al Ahli | 106842 | 5555806 | euros | 16465 |
| Asia | Arabia Saudita | Saudi Pro League | Al Ettifaq | 40740 | 2118462 | euros | 16468 |
| Asia | Arabia Saudita | Saudi

In [ ]:
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_response)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df3 = pd.DataFrame(matches)#, columns=[club | liga | pais | wyId | officialName | name | area_name | domestic_competition_name ])

df3 = df3[~df3.eq('---').all(axis=1)]
df3.columns = df3.iloc[0]  # La primera fila se convierte en nombres de columna
df3 = df3[1:]
# Resetea el índice
df3.reset_index(drop=True, inplace=True)

print(df3)

0                club                      liga            pais   wyId  \
0    Inhulets Petrove  Ukrainian Premier League         Ucrania  37291   
1               Minaj  Ukrainian Premier League         Ucrania  62921   
2           Rukh Lviv  Ukrainian Premier League         Ucrania  38304   
3             Al Ahli          Saudi Pro League  Arabia Saudita  16465   
4          Al Ettifaq          Saudi Pro League  Arabia Saudita  16468   
..                ...                       ...             ...    ...   
250               AIK               Allsvenskan          Suecia   6716   
251    Brommapojkarna               Allsvenskan          Suecia   6725   
252       Djurgardens               Allsvenskan          Suecia   6713   
253          Elfsborg               Allsvenskan          Suecia   6721   
254              GAIS               Allsvenskan          Suecia   6732   

0              officialName                    name     area_name  \
0    Sdyushor Mykolayiv U17  Sdyushor Myko

subo a BQ

In [ ]:
# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.teamidcapology'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(df3, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.teamidcapology.


In [70]:
prompt0 = f"""Sos un agente que analiza datos necesito que generes una tabla con la union de la informacion de dos tablas, a partir del analisis de estas 2 tablas.
La primera tabla contiene las columnas club, liga, pais de 570 clubes y la segunda tabla contiene las columnas wyId, officialName,name, area_name y domestic_competition_name con 28425 valores. Lo que yo necesito que hagas es, encontrar las coincidencias
entre todos los valores de las columnas y a partir de esas coincidencias obtener el valor del id en el campo "wyId" de la segunda tabla
estas son las tablas: {dftext} y {dftext2}.
Es decir solamente tienen que quedar en el resultado, los valores que coinciden dentro de las dos tablas con su ID, deberian ser la misma cantidad de valores que la tabla inicial es decir 570. La tabla resultado tiene que tener todos los campos.
Las coincidencias de no se exactas pueden ser parciales, por ejemplo aclaraciones:
El campo name u officialName con club, puede ser que en club el contenido se mas abreviado, pero si encontramos la palabra que coincide es valido.
El campo pais con area_name, tiene como diferencia el idioma asique puede ser que la coincidencia sea en otro idioma.
El campo liga, esta completo en la primera tabla, pero en la segunda no siempre esta completo.
La prioridad seria encontrar la coincidencia en el nombre del club, luego para continuar el match, que coincida con el pais o area_name, luego con la liga o domestic_competition.
y los que no se encuentra coincidencias, aclararlo tambien.
Es importante tener en cuenta que lo que queremos lograr con esto es obtener el wyId de la segunda tabla para que nos provea informacion a la primera tabla, en la cual no lo tenemos.
   """
# Generar la respuesta utilizando el prompt actualizado
responses0 = model.generate_content(prompt0, generation_config=generation_config, stream=True)

# Inicializar una variable para almacenar el resultado completo
full_response0 = ""

# Recopilar y unir todas las partes de la respuesta
for responses0 in responses0:
    full_response0 += responses0.text

# Imprimir o usar la respuesta completa almacenada en la variable
print(full_response0)

Entiendo perfectamente lo que necesitas. Para generar la tabla con la unión de la información de las dos tablas, necesito analizar cada fila de la primera tabla y buscar la coincidencia en la segunda tabla, teniendo en cuenta las prioridades que has establecido. 

**Aquí te presento un ejemplo de cómo se podría estructurar la tabla resultante:**

| Club | Liga | País | wyId | officialName | name | area_name | domestic_competition_name | region |
|---|---|---|---|---|---|---|---|---|
| Inhulets Petrove | Ukrainian Premier League | Ucrania | 14532 | FC Inhulets Petrove | Inhulets Petrove | Ukraine | Ukrainian Premier League | Europe |
| Minaj | Ukrainian Premier League | Ucrania | 14557 | FK Minaj | Minaj | Ukraine | Ukrainian Premier League | Europe |
| Rukh Lviv | Ukrainian Premier League | Ucrania | 14534 | FC Rukh Lviv | Rukh Lviv | Ukraine | Ukrainian Premier League | Europe |
| Al Ahli | Saudi Pro League | Arabia Saudita | 16465 | Al Ahli SC (Jeddah) | Al Ahli | Saudi Arabia | Saud

In [ ]:
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_response2)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df4 = pd.DataFrame(matches)#, columns=[club | liga | pais | wyId | officialName | name | area_name | domestic_competition_name ])

df4 = df4[~df4.eq('---').all(axis=1)]
df4.columns = df4.iloc[0]  # La primera fila se convierte en nombres de columna
df4 = df4[1:]
# Resetea el índice
df4.reset_index(drop=True, inplace=True)

print(df4)

In [ ]:
# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.2teamidcapology'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(df4, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.2teamidcapology.


In [76]:


# Supongamos que tus tablas están en archivos CSV
#df1 = pd.read_csv('tabla1.csv')
#df2 = pd.read_csv('tabla2.csv')

# Normalizar los nombres en ambas tablas
df1['club'] = df1['club'].str.lower().str.strip()
df1['liga'] = df1['liga'].str.lower().str.strip()
df1['pais'] = df1['pais'].str.lower().str.strip()
df1['region'] = df1['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['domestic_competition_name'] = df2['domestic_competition_name'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia en una lista
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 80 else None

# Encontrar el ID de los clubes en la segunda tabla
df1['wyId'] = df1['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))

# Merge basado en la columna 'wyId' para obtener la información completa
merged_df = pd.merge(df, df2, left_on='wyId', right_on='name', how='inner', suffixes=('_tabla1', '_tabla2'))

# Filtrar columnas y organizar el DataFrame final
final_columns = [
    'region_tabla1', 'pais_tabla1', 'liga_tabla1', 'club_tabla1', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'domestic_competition_name', 'region_tabla2'
]
final_df = merged_df[final_columns]

# Renombrar columnas para claridad
final_df.columns = [
    'region', 'pais', 'liga', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'domestic_competition_name', 'region'
]

# Guardar o imprimir el DataFrame final
final_df.to_csv('tabla_combinada.csv', index=False)
print(final_df)


KeyError: "['pais_tabla1', 'liga_tabla1', 'club_tabla1', 'wyId'] not in index"

In [77]:
# Verifica los nombres de las columnas en merged_df
print(merged_df.columns)


Index(['region_tabla1', 'pais', 'liga', 'club', 'weeklyavg', 'annualavg',
       'moneda', 'wyId_tabla1', 'wyId_tabla2', 'name', 'officialName',
       'area_name', 'domestic_competition_name', 'region_tabla2'],
      dtype='object')


UNION DE DF CON LOS PAISES EN INGLES DE LA TABLA SALARIOS_COPY CON DICT TEAMS USANDO FUZZY----


In [75]:
# Normalizar los nombres en ambas tablas
df22['club'] = df22['club'].str.lower().str.strip()
df22['liga'] = df22['liga'].str.lower().str.strip()
df22['pais'] = df22['pais'].str.lower().str.strip()
df22['region'] = df22['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['domestic_competition_name'] = df2['domestic_competition_name'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia en una lista
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 60 else None

# Encontrar el ID de los clubes en la segunda tabla
df22['wyId'] = df22['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df22['wyId'] = df22['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId' para obtener la información completa
merged_df = pd.merge(df22, df2, left_on='wyId', right_on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'liga', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'domestic_competition_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df2 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df2.rename(columns={
    'region_tabla1': 'region',
    'pais': 'pais',
    'liga': 'liga',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'domestic_competition_name': 'domestic_competition_name',
    'region_tabla2': 'region'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df2.to_csv('tabla_combinada.csv', index=False)
print(final_df2)


                   club   wyId                 name  \
0                  lafc    100                  afc   
1          orlando city   8068         orlando city   
2          nashville sc  30149         nashville sc   
3             dc united   7852            dc united   
4         new york city  32556        new york city   
5  sporting kansas city  35493      sporting canamy   
6    philadelphia union   8061   philadelphia union   
7                austin  58325            blaustein   
8   vancouver whitecaps  15015  vancouver whitecaps   
9         fc cincinnati  37892           cincinnati   

                  officialName  
0  amsterdamsche football club  
1              orlando city sc  
2                 nashville sc  
3                    dc united  
4             new york city fc  
5         club sporting canamy  
6           philadelphia union  
7                tsv blaustein  
8       vancouver whitecaps fc  
9                fc cincinnati  
                    region     

In [76]:
# Renombrar columnas duplicadas sino me da error al intentar subirlo a BQ
final_df2.columns = [
    'region_tabla1','pais','liga','club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name','domestic_competition_name' ,'region_tabla2'
]


In [77]:
from google.cloud import bigquery

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.fuzzy2'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(final_df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.fuzzy2.


sin liga

In [67]:
df22['club'] = df22['club'].str.lower().str.strip()
df22['pais'] = df22['pais'].str.lower().str.strip()
df22['region'] = df22['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia en una lista
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 70 else None

# Encontrar el ID de los clubes en la segunda tabla
df22['wyId'] = df22['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df22['wyId'] = df22['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId' para obtener la información completa
merged_df = pd.merge(df22, df2, on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df2 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df2.rename(columns={
    'region_tabla1': 'region',
    'pais': 'pais',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'region_tabla2': 'region'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df2.to_csv('tabla_combinada.csv', index=False)
print(final_df2)

                   club     wyId                 name  \
0                  lafc    100.0                  afc   
1          orlando city   8068.0         orlando city   
2          nashville sc  30149.0         nashville sc   
3             dc united   7852.0            dc united   
4         new york city  32556.0        new york city   
5  sporting kansas city  35493.0      sporting canamy   
6    philadelphia union   8061.0   philadelphia union   
7                austin  58325.0            blaustein   
8   vancouver whitecaps  15015.0  vancouver whitecaps   
9         fc cincinnati  37892.0           cincinnati   

                  officialName  
0  amsterdamsche football club  
1              orlando city sc  
2                 nashville sc  
3                    dc united  
4             new york city fc  
5         club sporting canamy  
6           philadelphia union  
7                tsv blaustein  
8       vancouver whitecaps fc  
9                fc cincinnati  
         

C:\Users\flori\AppData\Local\Temp\ipykernel_23592\771867300.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2.rename(columns={


In [72]:
# Renombrar columnas duplicadas sino me da error al intentar subirlo a BQ
final_df2.columns = [
    'region_tabla1', 'pais','club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'region_tabla2'
]


ValueError: Length mismatch: Expected axis has 11 elements, new values have 13 elements

In [74]:
from fuzzywuzzy import process, fuzz
import pandas as pd

# Normalizar las columnas
df22['club'] = df22['club'].str.lower().str.strip()
df22['pais'] = df22['pais'].str.lower().str.strip()
df22['region'] = df22['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 80 else None

# Inicializar una lista para almacenar los IDs
df22['wyId'] = None

# Priorizar los matchings
for column in ['club', 'pais', 'region']:
    if column == 'club':
        df22['wyId'] = df22['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))
    elif column == 'pais':
        # Solo buscar si no hay match en club
        df22.loc[df22['wyId'].isnull(), 'wyId'] = df22.loc[df22['wyId'].isnull(), 'pais'].apply(
            lambda x: get_best_match(x, df2['area_name'].tolist())
        )
    elif column == 'region':
        # Solo buscar si no hay match en pais
        df22.loc[df22['wyId'].isnull(), 'wyId'] = df22.loc[df22['wyId'].isnull(), 'region'].apply(
            lambda x: get_best_match(x, df2['region'].tolist())
        )

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df22['wyId'] = df22['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId'
merged_df = pd.merge(df22, df2, on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df2 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df2.rename(columns={
    'region_tabla1': 'region',
    'pais': 'pais',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'region_tabla2': 'region'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df2.to_csv('tabla_combinada.csv', index=False)
print(final_df2)


                   club   wyId                 name  \
0                  lafc    100                  afc   
1          orlando city   8068         orlando city   
2          nashville sc  30149         nashville sc   
3             dc united   7852            dc united   
4         new york city  32556        new york city   
5  sporting kansas city   8134        united states   
6    philadelphia union   8061   philadelphia union   
7                austin  58325            blaustein   
8   vancouver whitecaps  15015  vancouver whitecaps   
9         fc cincinnati  37892           cincinnati   

                  officialName  
0  amsterdamsche football club  
1              orlando city sc  
2                 nashville sc  
3                    dc united  
4             new york city fc  
5                united states  
6           philadelphia union  
7                tsv blaustein  
8       vancouver whitecaps fc  
9                fc cincinnati  
                    region     

C:\Users\flori\AppData\Local\Temp\ipykernel_23592\738396499.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2.rename(columns={


In [73]:
from google.cloud import bigquery

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.fuzzy'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(final_df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.fuzzy.
